In [18]:
import pandas as pd
from sqlalchemy import create_engine

## Extract CSVs into DataFrames

In [ ]:
population_file= "../Resources/population_figures_by_country.csv"
population_df= pd.read_csv(population_file)
population_df.head()